In [1]:
'''
%pip install pandas
%pip install scikit-learn
%pip install xgboost
'''

'\n%pip install pandas\n%pip install scikit-learn\n%pip install xgboost\n'

In [2]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor

## Data loading

In [3]:
df_bu_feat = pd.read_csv("data/bu_feat.csv.gz") 
df_train = pd.read_csv("data/train.csv.gz") 
df_test = pd.read_csv("data/test.csv.gz")

### Merging features

In [4]:
df_train_feat = pd.merge(df_train, df_bu_feat, how="left", on = "but_num_business_unit")
df_test_feat = pd.merge(df_test, df_bu_feat, how="left", on = "but_num_business_unit")

### Split train, val set

In [5]:
df_train_feat

,day_id,but_num_business_unit,dpt_num_department,turnover,but_postcode,but_latitude,but_longitude,but_region_idr_region,zod_idr_zone_dgr
0,2017-09-30,64,127,580.308443,16400,45.625172,0.111939,70,10
1,2017-09-30,119,127,1512.995918,74100,46.195037,6.254448,51,4
2,2017-09-30,4,88,668.593556,6600,43.600994,7.078160,55,10
3,2017-09-30,425,127,0.000000,59000,50.617921,3.084186,33,3
4,2017-09-30,513,73,0.000000,33610,44.717366,-0.733429,33,3
...,...,...,...,...,...,...,...,...,...
277714,2012-12-29,131,73,1.461821,69760,45.822363,4.767741,8,4
277715,2012-12-29,237,127,759.283046,13500,43.430995,5.047929,71,10
277716,2012-12-29,129,117,1716.399152,14124,49.154936,-0.287441,30,6
277717,2012-12-29,468,127,1307.357057,94320,48.757857,2.385381,75,6


In [6]:
# Train and val set

df_train_feat["day_id"] = pd.to_datetime(df_train_feat["day_id"])
df_train_feat["day_id_week"] = df_train_feat.day_id.dt.isocalendar().week
df_train_feat["day_id_month"] = df_train_feat["day_id"].dt.month
df_train_feat["day_id_year"] = df_train_feat["day_id"].dt.year

df_train = df_train_feat[(df_train_feat.day_id_year < 2017)]
df_val = df_train_feat[(df_train_feat.day_id_year == 2017)]

y_train = df_train.turnover
y_val = df_val.turnover


### Scikit pipeline

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

In [8]:
class CustomPreprocressing(BaseEstimator, TransformerMixin):
    """
    This class includes all the steps for the preprocessing
    """
    def __init__(self, cat_cols):
        """
        Initialize the class / Can be empty
        """
        self.cat_cols = cat_cols

    def fit(self, X, y=None):
        """
        This method is only created so that the pipeline containing this transformer does not raise an error
        """
        return self

    def transform(self, data):
        """
        Inputs :
          -- data : DataFrame, DataFrame contening all the data needed for the model
        Outputs :
          -- DataFrame, DataFrame prepared for modeling

        """
        data["day_id"] = pd.to_datetime(data["day_id"])
        data["day_id_week"] = data.day_id.dt.isocalendar().week
        data["day_id_month"] = data["day_id"].dt.month
        data["day_id_year"] = data["day_id"].dt.year
        data[self.cat_cols] = data[self.cat_cols].apply(lambda x: x.astype(str))
        return data

In [9]:

num_attrib = ["but_latitude","but_longitude", 'day_id_year']
cat_attrib = [
            "day_id_week",
            "day_id_month",
            "but_region_idr_region",
            "zod_idr_zone_dgr",
            "but_num_business_unit",
            "dpt_num_department",
        ]

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])
cat_onehot_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown="ignore")),
])
preparation_pipeline = ColumnTransformer([
    ("num",num_pipeline, num_attrib),
    ("cat_onehot", cat_onehot_pipeline, cat_attrib)
])

full_pipeline = Pipeline([
    ('preprocessing', CustomPreprocressing(cat_cols=cat_attrib )),
    ('preparation', preparation_pipeline),
    ('model', GradientBoostingRegressor())
])

In [10]:
df_val

,day_id,but_num_business_unit,dpt_num_department,turnover,but_postcode,but_latitude,but_longitude,but_region_idr_region,zod_idr_zone_dgr,day_id_week,day_id_month,day_id_year
0,2017-09-30,64,127,580.308443,16400,45.625172,0.111939,70,10,39,9,2017
1,2017-09-30,119,127,1512.995918,74100,46.195037,6.254448,51,4,39,9,2017
2,2017-09-30,4,88,668.593556,6600,43.600994,7.078160,55,10,39,9,2017
3,2017-09-30,425,127,0.000000,59000,50.617921,3.084186,33,3,39,9,2017
4,2017-09-30,513,73,0.000000,33610,44.717366,-0.733429,33,3,39,9,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
48763,2017-01-07,97,117,1440.429516,62210,50.407462,2.972838,69,4,1,1,2017
48764,2017-01-07,83,127,1688.739694,42390,45.468710,4.343640,8,4,1,1,2017
48765,2017-01-07,1121,88,43.963324,38230,45.747941,5.183154,178,72,1,1,2017
48766,2017-01-07,218,127,806.474037,31150,43.687736,1.404662,64,10,1,1,2017


In [11]:
model_final = full_pipeline.fit(df_train, y_train)
y_predict_val = model_final.predict(df_val)

metric_mae = mean_absolute_error(y_val, y_predict_val)
print(metric_mae)

/var/folders/dt/0rph8wwj0ld_0cv19w9m4w900000gn/T/ipykernel_74591/3655981615.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["day_id"] = pd.to_datetime(data["day_id"])
/var/folders/dt/0rph8wwj0ld_0cv19w9m4w900000gn/T/ipykernel_74591/3655981615.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["day_id_week"] = data.day_id.dt.isocalendar().week
/var/folders/dt/0rph8wwj0ld_0cv19w9m4w900000gn/T/ipykernel_74591/3655981615.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

407.2289311686874


In [12]:
df_val

,day_id,but_num_business_unit,dpt_num_department,turnover,but_postcode,but_latitude,but_longitude,but_region_idr_region,zod_idr_zone_dgr,day_id_week,day_id_month,day_id_year
0,2017-09-30,64,127,580.308443,16400,45.625172,0.111939,70,10,39,9,2017
1,2017-09-30,119,127,1512.995918,74100,46.195037,6.254448,51,4,39,9,2017
2,2017-09-30,4,88,668.593556,6600,43.600994,7.078160,55,10,39,9,2017
3,2017-09-30,425,127,0.000000,59000,50.617921,3.084186,33,3,39,9,2017
4,2017-09-30,513,73,0.000000,33610,44.717366,-0.733429,33,3,39,9,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
48763,2017-01-07,97,117,1440.429516,62210,50.407462,2.972838,69,4,1,1,2017
48764,2017-01-07,83,127,1688.739694,42390,45.468710,4.343640,8,4,1,1,2017
48765,2017-01-07,1121,88,43.963324,38230,45.747941,5.183154,178,72,1,1,2017
48766,2017-01-07,218,127,806.474037,31150,43.687736,1.404662,64,10,1,1,2017


In [13]:
df_val['dpt_num_department'].unique()

array(['127', '88', '73', '117'], dtype=object)

In [14]:
df_val['prediction'] = y_predict_val
filter_ts = lambda x: (x.but_num_business_unit=="32") & (x.dpt_num_department=='73')
display(df_val[filter_ts])

display(df_train[filter_ts])

/var/folders/dt/0rph8wwj0ld_0cv19w9m4w900000gn/T/ipykernel_74591/1857063852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['prediction'] = y_predict_val


,day_id,but_num_business_unit,dpt_num_department,turnover,but_postcode,but_latitude,but_longitude,but_region_idr_region,zod_idr_zone_dgr,day_id_week,day_id_month,day_id_year,prediction
1089,2017-09-30,32,73,452.769296,29490,48.425054,-4.436446,7,6,39,9,2017,469.318872
2519,2017-09-23,32,73,513.263277,29490,48.425054,-4.436446,7,6,38,9,2017,469.318872
3432,2017-09-16,32,73,912.694843,29490,48.425054,-4.436446,7,6,37,9,2017,469.318872
4290,2017-09-09,32,73,831.668945,29490,48.425054,-4.436446,7,6,36,9,2017,469.318872
5599,2017-09-02,32,73,528.774473,29490,48.425054,-4.436446,7,6,35,9,2017,469.318872
6357,2017-08-26,32,73,562.253678,29490,48.425054,-4.436446,7,6,34,8,2017,502.036028
7705,2017-08-19,32,73,898.535514,29490,48.425054,-4.436446,7,6,33,8,2017,502.036028
9811,2017-08-12,32,73,1049.221014,29490,48.425054,-4.436446,7,6,32,8,2017,502.036028
10973,2017-08-05,32,73,1082.103732,29490,48.425054,-4.436446,7,6,31,8,2017,502.036028
12303,2017-07-29,32,73,1155.885670,29490,48.425054,-4.436446,7,6,30,7,2017,502.036028


,day_id,but_num_business_unit,dpt_num_department,turnover,but_postcode,but_latitude,but_longitude,but_region_idr_region,zod_idr_zone_dgr,day_id_week,day_id_month,day_id_year
49581,2016-12-31,32,73,544.941047,29490,48.425054,-4.436446,7,6,52,12,2016
50950,2016-12-24,32,73,1023.650512,29490,48.425054,-4.436446,7,6,51,12,2016
51720,2016-12-17,32,73,1231.944726,29490,48.425054,-4.436446,7,6,50,12,2016
53122,2016-12-10,32,73,657.439375,29490,48.425054,-4.436446,7,6,49,12,2016
54834,2016-12-03,32,73,715.383766,29490,48.425054,-4.436446,7,6,48,12,2016
...,...,...,...,...,...,...,...,...,...,...,...,...
272807,2013-01-26,32,73,317.378897,29490,48.425054,-4.436446,7,6,4,1,2013
274473,2013-01-19,32,73,539.422889,29490,48.425054,-4.436446,7,6,3,1,2013
275216,2013-01-12,32,73,475.406553,29490,48.425054,-4.436446,7,6,2,1,2013
275962,2013-01-05,32,73,229.110309,29490,48.425054,-4.436446,7,6,1,1,2013


In [15]:
model_final = full_pipeline.fit(df_train_feat, df_train_feat.turnover.values)

In [16]:
y_pred = model_final.predict(df_test_feat)

In [17]:
df_test_feat['prediction'] = y_pred

In [18]:
display(df_test_feat)

,day_id,but_num_business_unit,dpt_num_department,but_postcode,but_latitude,but_longitude,but_region_idr_region,zod_idr_zone_dgr,day_id_week,day_id_month,day_id_year,prediction
0,2017-11-25,95,73,80000,49.869382,2.280452,69,4,47,11,2017,28.096508
1,2017-11-25,4,117,6600,43.600994,7.078160,55,10,47,11,2017,324.086069
2,2017-11-25,113,127,84014,43.919562,4.867583,115,10,47,11,2017,1333.614589
3,2017-11-25,93,117,13008,43.239744,5.396694,71,10,47,11,2017,357.205467
4,2017-11-25,66,127,34500,43.347835,3.255024,6,10,47,11,2017,1389.478844
...,...,...,...,...,...,...,...,...,...,...,...,...
10131,2017-10-07,61,88,60740,49.238738,2.468513,69,4,40,10,2017,259.168393
10132,2017-10-07,641,117,17810,45.749749,-0.675981,70,10,40,10,2017,241.856927
10133,2017-10-07,724,117,38150,45.327709,4.804922,52,4,40,10,2017,241.856927
10134,2017-10-07,1302,117,74950,46.069548,6.549448,51,4,40,10,2017,324.086069


##### Model Serialization

In [19]:
import joblib 
# Save the model
joblib.dump(model_final,"../turnover_forecasting_model.pkl")

['../turnover_forecasting_model.pkl']

##### Model Loading

In [20]:
model = joblib.load('../turnover_forecasting_model.pkl')

##### Testing Prediction

In [21]:
from pydantic import BaseModel
from datetime import datetime

class DonneesEntree(BaseModel):
    but_num_business_unit: int  # store's ID
    dpt_num_department: int  # department's ID
    but_postcode: int  # postal code
    but_latitude: float  # store's latitude
    but_longitude: float  # store's longitude
    but_region_idr_region: int  # region's number
    zod_idr_zone_dgr: int  # zone's number
    day_id : datetime # date

request={"but_num_business_unit":100,
              "dpt_num_department":112,
              "but_postcode":75000,
              "but_latitude":43.34,
              "but_longitude":34.83,
              "but_region_idr_region":23,
              "zod_idr_zone_dgr":10,
              "day_id":2017-11-25}

donnees = DonneesEntree(**request)
donnees_df = pd.DataFrame([donnees.dict()])  # Conversion en DataFrame
donnees_df["day_id"] = pd.to_datetime(donnees_df["day_id"])
donnees_df["day_id_week"] = donnees_df.day_id.dt.isocalendar().week
donnees_df["day_id_month"] = donnees_df["day_id"].dt.month
donnees_df["day_id_year"] = donnees_df["day_id"].dt.year
int(model.predict(donnees_df)[0])

1334